# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

Recall Problem 1 from hw3t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [2]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [3]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [4]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [5]:
def bfs(graph, s, t):
    explored, queue = [s], [(s, [s])]
    while queue:
        current = queue[0]
        queue = queue[1:]
        node, path = current[0], current[1]
        for node in graph.neighbors(node):
            if node in explored:
                continue
            if node == t:
                return path + [node]
            if node not in explored:
                queue.append((node, path + [node]))
                explored.append(node)

In [6]:
def find_bottleneck(graph, path):
    bottleneck = float('inf')
    for i in range(len(path)-1):
        if bottleneck > graph.edges[path[i], path[i+1]]['capacity']:
            bottleneck = graph.edges[path[i], path[i+1]]['capacity']
    return bottleneck

In [7]:
def myFF(graph, s, t):
    G_c = graph.copy()
    path = bfs(graph, s, t)
    flow = 0
    
    # updating residual graph, finding maxflow
    while path:
        bottleneck = find_bottleneck(G_c, path)
        flow += bottleneck
        
        for i in range(len(path)-1):
            if G_c.edges[path[i], path[i+1]]['capacity'] == bottleneck:
                G_c.remove_edge(path[i], path[i+1])
            else:
                G_c.edges[path[i], path[i+1]]['capacity'] -= bottleneck
                
        path = bfs(G_c, s, t)
    
    # finding the flow through each edge
    for (s1, s2) in graph.edges():
        if not G_c.has_edge(s1,s2):
            G_c.add_edge(s1, s2)
            G_c.edges[s1, s2]['capacity'] = graph.edges[s1, s2]['capacity']
        else:
            G_c.edges[s1, s2]['capacity'] = graph.edges[s1, s2]['capacity'] - G_c.edges[s1, s2]['capacity']
    
    flow_dict = {}
    for node in G_c.nodes():
        flow_dict[node] = {}
    for node in G_c.nodes():
        for neighbor in G_c.neighbors(node):
            flow_dict[node][neighbor] = G_c.edges[node, neighbor]['capacity']
            
    return (flow, flow_dict)
            

In [8]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    # TODO: Implement the function.
    graph_c = graph.copy()
    graph_c.add_node('s')
    graph_c.add_node('t')

    supply, demand = 0, 0
    for node in graph_c.nodes():
        if node in ['s', 't']:
            continue
        elif graph_c.nodes[node]['demand'] < 0:
            graph_c.add_edge('s', node, capacity= -graph_c.nodes[node]['demand'])
            supply += -graph_c.nodes[node]['demand']
        else:
            graph_c.add_edge(node, 't', capacity= graph_c.nodes[node]['demand'])
            demand += graph_c.nodes[node]['demand']
    
    flow = myFF(graph_c, 's', 't')[0]
    flow_dict = myFF(graph_c, 's', 't')[1]

    if supply != demand or flow != demand:
        raise nx.NetworkXUnfeasible("there is no flow satisfying the demands")

    flow_dict.pop('s', None)
    flow_dict.pop('t', None)
    for i in flow_dict:
        flow_dict[i].pop('s', None)
        flow_dict[i].pop('t', None)
    
    return flow_dict

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [9]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    # TODO: Implement the function.
    
    node_flow_dict = {}
    for node in flow:
        node_flow_dict[node] = 0

    for i in flow:
        for j in flow[i]:
            node_flow_dict[i] -= flow[i][j]
            node_flow_dict[j] += flow[i][j]
            
    return node_flow_dict

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [10]:
flow = flow_with_demands(G)
div = divergence(flow)
#print ("Flow satisfies all demands:", all(div[n] == G.nodes[n]['demand'] for n in G.nodes()))